# Crawler

This notebook contains started code structure for creating a crawler on single machine

**Author:** Noshaba Nasir

**Date:** 26/3/2021

**Updated by:** <Write your name and Roll number here>
    

In [1]:
import os 
import codecs 
import random
import requests
import numpy as np
from queue import Queue
from datetime import datetime,timedelta
import threading, queue, time, urllib
from bs4 import BeautifulSoup
from urllib import request
from urllib.parse import urlsplit
from urllib.parse import urlparse
from urllib import parse
from urllib import robotparser
# Add any library to be imported here

In [2]:
# Crawler Parameters
BACKQUEUES= 3
THREADS= BACKQUEUES*3
FRONTQUEUES= 5
WAITTIME= 15 ; # wait 15 seconds before fetching URLS from 
FILENAME = 1

# Add any other global parameters here

# FRONTIER

Frontier should use the Mercator frontier design as discussed in lecture.

Preferably it should be a class and should have the given functions.

*prioritizer* function is a stub right now, it will return a random number  between 1 to f for given URL 

In [3]:
def date_diff_in_Seconds(dt2, dt1):
    timedelta = dt2 - dt1
    return timedelta.days * 24 * 3600 + timedelta.seconds

In [4]:
class frontier:
# add the code for frontier here
# should have functions __init__, get_URL, add_URLs, add_to_backqueue

    def __init__(self,FQueue,BQueue,TimeStamp,CrawledURL,TotalUrlCrawled):
        self.FQueue = FQueue
        self.BQueue = BQueue
        self.TimeStamp = TimeStamp
        self.CrawledURL = CrawledURL
        self.TotalUrlCrawled = TotalUrlCrawled
        seed_urls = [
            "https://docs.oracle.com/en/",
            "https://www.oracle.com/corporate/",
            "https://en.wikipedia.org/wiki/Machine_learning",
            "https://www.csie.ntu.edu.tw/~cjlin/libsvm/index.html",
            "https://docs.oracle.com/middleware/jet210/jet/index.html",
            "https://en.wikipedia.org/w/api.php",
            "https://en.wikipedia.org/api/",
            "https://en.wikipedia.org/wiki/Weka_(machine_learning)"
        ]
        for i in range(len(seed_urls)):
            priority = random.randint(0,4)
            FQueue[priority].put(seed_urls[i])
        return
    #Adds URL to corresponding Front Queue after giving it prioirity using prioritizer function 
    def add_URLs(self,URL,FRONTQUEUES):
        priority = self.prioritizer(URL,FRONTQUEUES)
        #print(FQueue[priority].qsize())
        FQueue[priority].put(URL)
        #print(FQueue[priority].qsize())
        return
    
    def prioritizer(self,URL,f):
        return random.randint(0,f-1)
    
    # Returns BackQueue which has not been accsessed for the longest time
    def get_URL(self):
        CurrentDateTime=datetime.now()
        TimeDiff =[]
        TimeDiff = [0 for i in range(3)] 
#         print("TIMESTAMP")
#         print(self.TimeStamp[1])
        for i in range(3):
            TimeDiff[i]=date_diff_in_Seconds(CurrentDateTime, self.TimeStamp[i])
        
        MinBackQueue = TimeDiff.index(max(TimeDiff))
        print(TimeDiff)
        return MinBackQueue
        
    def add_to_backqueue(self):
        for i in range(3):
            if(BQueue[i].qsize() == 0): #BackQueue is empty
                FrontQueue = np.random.choice(np.arange(0, 5), p=[(1/15), (2/15), (3/15), (4/15), (5/15)])
                while(FQueue[FrontQueue].qsize() == 0):
                    FrontQueue = np.random.choice(np.arange(0, 5), p=[(1/15), (2/15), (3/15), (4/15), (5/15)])
                URL= FQueue[FrontQueue].queue[0]
                print(URL)
                parts = urlsplit(URL)
                base_url = "{0.scheme}://{0.netloc}".format(parts)
                
                flag = 0
                for j in range(3):
                    if BQueue[j].qsize() != 0:
                        Backqueue_URL=BQueue[j].queue[0]
                        Backqueue_parts = urlsplit(Backqueue_URL)
                        Backqueue_base_url = "{0.scheme}://{0.netloc}".format(Backqueue_parts)
                        
                        if Backqueue_base_url == base_url : # Backqueue baseURL match base_url in another queue
                            flag = 1
                            BQueue[j].put(FQueue[FrontQueue].get()) #URL to backqueue as base url match
                if(flag == 0):
                    #print(FQueue[FrontQueue].qsize())
                    BQueue[i].put(FQueue[FrontQueue].get())
                    #print(FQueue[FrontQueue].qsize())
                    return True
                else:
                    #print("FLAG")
                    return False
        
        

In [5]:
FQueue = [ Queue() for i in range(FRONTQUEUES) ]
BQueue = [ Queue() for i in range(BACKQUEUES) ]
TimeStamp = []
TimeStamp = [datetime.now() - timedelta(seconds=15) for i in range(3)] 
CrawledURL=set()
TotalUrlCrawled = 0

Object = frontier(FQueue,BQueue,TimeStamp,CrawledURL,TotalUrlCrawled)



# FILTER URLS

Filter the URLS that are in robots.txt files of server and the have been already processed. 

In [6]:
# write code for filtering urls here 
def filter_robot(base_URL, Crawled_URLs):
    Filtered_URLs = []
    size = len(Crawled_URLs)
    for i in range (size):
        parser = robotparser.RobotFileParser()
        parser.set_url(parse.urljoin(base_URL, 'robots.txt'))
        parser.read()
        flag = parser.can_fetch("*",Crawled_URLs[i])
        if(flag == True):
            Filtered_URLs.append(Crawled_URLs[i])
    return Filtered_URLs
    

## ---------------------Write any other codes/data require to run the crawler above this cell-----------

# Run Crawler

In [7]:
# Thread task
def crawler_thread_task():
    
    # If any backqueue is empty populate it using add to backqueue()
    lock.acquire()
    while(Object.BQueue[0].qsize() == 0 or Object.BQueue[1].qsize() == 0 or Object.BQueue[2].qsize() == 0 ):
        Object.add_to_backqueue()
    lock.release()
    # get Back queue number whose last access time is maximum
    BQueueNumber = Object.get_URL()

   
    #BackQueue with least time selected is checked if the server was reached less than 15 seconds ago if yes trhe thread sleeps
    CurrentDateTime = datetime.now()
    LastAccessed = date_diff_in_Seconds(CurrentDateTime, Object.TimeStamp[BQueueNumber])
    while(LastAccessed < 15):
        time.sleep(1)
        CurrentDateTime=datetime.now()
        LastAccessed = date_diff_in_Seconds(CurrentDateTime, Object.TimeStamp[BQueueNumber])
        
    lock.acquire()
    #URL which needs to be processed
    ProcessURL = BQueue[BQueueNumber].get() 
    #Update last access time of back queue
    Object.TimeStamp[BQueueNumber] = datetime.now()
    print("BackQueue Number:")
    print(BQueueNumber)
    lock.release()
    
    print("URL PROCESSING NOW")
    print(ProcessURL)
    ProcessedURLs.add(ProcessURL)

    soup = BeautifulSoup(requests.get(url=ProcessURL,).text, 'lxml')
#     with open("output1.html", "a", encoding='utf-8') as file:
#         file.write(str(soup))
    global FILENAME
    f = codecs.open('example'+str(FILENAME)+'.xml','a','utf-8')
    f.write(str(soup))
    f.close()
    FILENAME = FILENAME+1
    parts = urlsplit(ProcessURL)
    base = "{0.netloc}".format(parts)
    strip_base = base.replace("www.", "")
    base_url = "{0.scheme}://{0.netloc}".format(parts)
    path = ProcessURL[:ProcessURL.rfind('/')+1] if '/' in parts.path else ProcessURL
    for link in soup.find_all('a'):
        anchor = link.attrs["href"] if "href" in link.attrs else ''
        if anchor.startswith('/'):
            Crawled_URLs.append(base_url + anchor) #relative URL
        elif anchor.startswith('http'):
            Crawled_URLs.append(path + anchor) #absolute URL
    Filtered_URL =[]
    #Only permitted URL returned
    Filtered_URL = filter_robot(base_url,Crawled_URLs)
    UniqueCrawled =set(Filtered_URL) # Only unique crawled URLs
    # ensures the URL already processed do not get added to the front queue again
    FinalURL = UniqueCrawled.difference(ProcessedURLs)  # Add to frontier as Filtered and Duplicates removed
    
    print(ProcessedURLs)
    print("Crawled")
    print(len(Crawled_URLs))
    print("Filtered")
    print(len(Filtered_URL))
    print("FINALURLS")
    print(len(FinalURL))
    lock.acquire()
    Object.TotalUrlCrawled = Object.TotalUrlCrawled+len(FinalURL)
    lock.release()
    f = open("CrawledURLs.txt", "a")
    for i in range(len(FinalURL)):
        url = FinalURL.pop()
        Object.add_URLs(url,FRONTQUEUES)
        Object.CrawledURL.add(url)
        f.write(url+"\n")
    f.close()
    return # define individual crawler thread's function here as studies in class

In [8]:
def reapeatFunction():
    while Object.TotalUrlCrawled < 1000:
        crawler_thread_task()
        print("Number of URL's Crawled ")
        print(Object.TotalUrlCrawled)
    return

In [9]:
# intialize every thing 
lock = threading.Lock()
ProcessedURLs= set()
Crawled_URLs = []
Filtered_URL =[]
FinalURL = set()

In [10]:
# start the threads
#crawler_thread_task()
for i in range(THREADS):
    t = threading.Thread(target=reapeatFunction)
    t.start()
    t.join() 
print("Program Terminated")     


https://en.wikipedia.org/wiki/Weka_(machine_learning)
https://docs.oracle.com/en/
https://docs.oracle.com/middleware/jet210/jet/index.html
https://en.wikipedia.org/wiki/Machine_learning
https://en.wikipedia.org/w/api.php
https://www.oracle.com/corporate/
[15, 15, 15]
BackQueue Number:
0
URL PROCESSING NOW
https://en.wikipedia.org/wiki/Weka_(machine_learning)
{'https://en.wikipedia.org/wiki/Weka_(machine_learning)'}
Crawled
223
Filtered
199
FINALURLS
179
Number of URL's Crawled 
179
[149, 164, 164]
BackQueue Number:
1
URL PROCESSING NOW
https://docs.oracle.com/en/
{'https://docs.oracle.com/en/', 'https://en.wikipedia.org/wiki/Weka_(machine_learning)'}
Crawled
223
Filtered
223
FINALURLS
203
Number of URL's Crawled 
382
[501, 351, 516]
BackQueue Number:
2
URL PROCESSING NOW
https://www.oracle.com/corporate/
{'https://www.oracle.com/corporate/', 'https://docs.oracle.com/en/', 'https://en.wikipedia.org/wiki/Weka_(machine_learning)'}
Crawled
264
Filtered
264
FINALURLS
241
Number of URL's Cra

## ------------------------------------------------------End of Notebook---------------------------------------------------